### Saturday, December 2, 2023

Re ran this with the "TheBloke/CodeLlama-34B-Instruct-GPTQ" model, and it works! Nice! 20298MiB VRAM!

### Friday, November 17, 2023

Been doing some model cleanup on the hfpt_Oct28 container, and want to re-run this just to ensure all is still well ...

Yup! This still works! Nice!

So THIS notebook was just copied from TheBlokeCodeLlama.ipynb and I want to use this to download the 13B model and compare it against the 34B model ...

So the 13B model has been downloaded and works just fine! NOTE THIS NOTEBOOK DOES NOT USE LANGCHAIN!

### Thursday, November 16, 2023

https://huggingface.co/TheBloke/CodeLlama-34B-Instruct-GPTQ

Let's see if we can get this to run locally, shall we ... 

I am trying this model because it has the most downloads of all his models, AND of all the huggingface models that are 'code' related.

Wow! This actually works on the 4090! I am stunned! 20524MiB of VRAM!


In [ ]:
# docker cp c9b676310ea0://home/rob/Data2/huggingface/transformers/models--TheBloke--CodeLlama-34B-Instruct-GPTQ /home/rob/Data3/huggingface/transformers
# Successfully copied 18.3GB to /home/rob/Data3/huggingface/transformers

# docker cp c9b676310ea0://home/rob/Data2/huggingface/transformers/models--TheBloke--CodeLlama-13B-Instruct-GPTQ /home/rob/Data3/huggingface/transformers
# Successfully copied 7.26GB to /home/rob/Data3/huggingface/transformers

# (base) rob@KAUWITB:~$ docker cp /home/rob/Data3/huggingface/transformers/models--TheBloke--CodeLlama-34B-Instruct-GPTQ c9b676310ea0://home/rob/Data2/huggingface/transformers
# Successfully copied 18.3GB to c9b676310ea0://home/rob/Data2/huggingface/transformers
# (base) rob@KAUWITB:~$ 


In [4]:
# You need this to just target the 4090, othewise the 34B model does NOT load.
# only target the 4090 ...
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"

Started with all of these models below ...

In [5]:
!ls /home/rob/Data2/huggingface/transformers

models--EleutherAI--gpt-neox-20b	     models--mosaicml--mpt-7b-instruct
models--codellama--CodeLlama-7b-hf	     tmp_35pwa_q
models--meta-llama--Llama-2-13b-chat-hf      tmphxgsvifp
models--meta-llama--Llama-2-13b-hf	     tmpjiz9wrho
models--meta-llama--Llama-2-7b-chat-hf	     tmppceppekl
models--meta-llama--Llama-2-7b-hf	     tmpt7cv3q8c
models--mistralai--Mistral-7B-Instruct-v0.1  version.txt
models--mistralai--Mistral-7B-v0.1


But I am going to whack them all ... The 2TB disk, where we store the docker images and containers, is currently at 1.3TB free (34.9% full)

In [11]:
!rm -rf /home/rob/Data2/huggingface/transformers

In [12]:
!ls /home/rob/Data2/huggingface/transformers

ls: cannot access '/home/rob/Data2/huggingface/transformers': No such file or directory


Make the transformers directory ...

In [13]:
!mkdir /home/rob/Data2/huggingface/transformers

In [14]:
!ls /home/rob/Data2/huggingface/transformers

And now we are at 1.4TB free (27.6% free)

Now copy in just the model we want to work with, "TheBloke/CodeLlama-34B-Instruct-GPTQ" ...

(base) rob@KAUWITB:~$ docker cp /home/rob/Data3/huggingface/transformers/models--TheBloke--CodeLlama-34B-Instruct-GPTQ c9b676310ea0://home/rob/Data2/huggingface/transformers
Successfully copied 18.3GB to c9b676310ea0://home/rob/Data2/huggingface/transformers




In [15]:
!ls /home/rob/Data2/huggingface/transformers

models--TheBloke--CodeLlama-34B-Instruct-GPTQ


We are now at 1.4TB free, 28.5% full ... I am really glad I cloned that 1TB drive to this 2TB drive!

In [20]:
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

model_name_or_path = "TheBloke/CodeLlama-34B-Instruct-GPTQ"

# Let's go with the smaller model, shall we ... 
# model_name_or_path = "TheBloke/CodeLlama-13B-Instruct-GPTQ"

# # To use a different branch, change revision
# # For example: revision="main"
# model = AutoModelForCausalLM.from_pretrained(model_name_or_path,
#                                              device_map="auto",
#                                              trust_remote_code=False,
#                                              revision="main")

tokenizer = AutoTokenizer.from_pretrained(model_name_or_path, use_fast=True)

In [17]:
%%time
# To use a different branch, change revision
# For example: revision="main"
model = AutoModelForCausalLM.from_pretrained(model_name_or_path,
                                             device_map="auto",
                                             trust_remote_code=False,
                                             revision="main")

# 16.7s to load, takes up 20132Mib VRAM

# First time to download ...
# model_name_or_path = "TheBloke/CodeLlama-13B-Instruct-GPTQ"
# CPU times: user 43.8 s, sys: 43.2 s, total: 1min 26s
# Wall time: 1h 42min 31s

You have activated exllama backend. Note that you can get better inference
                    speed using exllamav2 kernel by setting `use_exllama_v2=True`.`disable_exllama` will be deprecated
                    in future version.


CPU times: user 26.5 s, sys: 12.5 s, total: 39 s
Wall time: 16 s


In [18]:
# from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

# model_name_or_path = "TheBloke/CodeLlama-34B-Instruct-GPTQ"
# # To use a different branch, change revision
# # For example: revision="main"
# model = AutoModelForCausalLM.from_pretrained(model_name_or_path,
#                                              device_map="auto",
#                                              trust_remote_code=False,
#                                              revision="main")

# tokenizer = AutoTokenizer.from_pretrained(model_name_or_path, use_fast=True)

prompt = "Tell me about AI"
prompt_template=f'''[INST] Write code to solve the following coding problem that obeys the constraints and passes the example test cases. Please wrap your code answer using ```:
{prompt}
[/INST]

'''

print("\n\n*** Generate:")

input_ids = tokenizer(prompt_template, return_tensors='pt').input_ids.cuda()
output = model.generate(inputs=input_ids, temperature=0.7, do_sample=True, top_p=0.95, top_k=40, max_new_tokens=512)
print(tokenizer.decode(output[0]))

# # Inference can also be done using transformers' pipeline

# print("*** Pipeline:")
# pipe = pipeline(
#     "text-generation",
#     model=model,
#     tokenizer=tokenizer,
#     max_new_tokens=512,
#     do_sample=True,
#     temperature=0.7,
#     top_p=0.95,
#     top_k=40,
#     repetition_penalty=1.1
# )

# print(pipe(prompt_template)[0]['generated_text'])

# 17.3s 10128MiB VRAM

# 22.5s 20524Mib VRAM




*** Generate:
<s> [INST] Write code to solve the following coding problem that obeys the constraints and passes the example test cases. Please wrap your code answer using ```:
Tell me about AI
[/INST]

The task is to write a program that generates a description of AI.

Here's an example of how the program might work:

```
#include <iostream>
#include <string>

using namespace std;

int main() {
  string description = "Artificial intelligence (AI) refers to the ability of a computer or machine to perform tasks that typically require human intelligence, such as understanding natural language, recognizing images, making decisions, and solving problems. AI systems use a combination of algorithms and statistical models to analyze data and make predictions or decisions. The field of AI is rapidly growing, with applications in industries such as healthcare, finance, and transportation.";

  cout << description << endl;

  return 0;
}
```

This program generates a description of AI using a s

In [19]:
# Inference can also be done using transformers' pipeline

print("*** Pipeline:")
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=512,
    do_sample=True,
    temperature=0.7,
    top_p=0.95,
    top_k=40,
    repetition_penalty=1.1
)

print(pipe(prompt_template)[0]['generated_text'])

# 10.1s 10128MiB VRAM

# 11.9s 20524Mib VRAM

*** Pipeline:
[INST] Write code to solve the following coding problem that obeys the constraints and passes the example test cases. Please wrap your code answer using ```:
Tell me about AI
[/INST]

Hello! I'm just an AI, my purpose is to assist users with information and tasks. I am trained on a wide range of texts and can generate human-like responses to questions and prompts. My capabilities include understanding natural language, generating text, and answering questions. I can be used in a variety of applications such as customer service, language translation, and content generation. However, it's important to note that I am not perfect and may make mistakes or not fully understand the context of certain questions or prompts.
